In [56]:
from backoff import on_exception, expo
from collections import defaultdict
import numpy as np
import pandas as pd
import requests
from ratelimit import limits, sleep_and_retry

In [303]:
input_df = pd.read_csv('../data/conmed_example_data_with_best_rxcuid.csv').fillna(np.nan).replace([np.nan], [None])

In [304]:
input_df['Best RxNorm Id'] = [str(int(rxcui)) if rxcui else None for rxcui in df['Best RxNorm Id'] ]

In [289]:
base_api_url = 'https://rxnav.nlm.nih.gov/REST'

@sleep_and_retry
@on_exception(expo, ConnectionError, max_tries=10)
@limits(calls=20, period=2)
def call_class_api(cui):
    api_url = '{}/rxclass/class/byRxcui.json?rxcui={}'.format(
        base_api_url, rxcui)
    response = requests.get(api_url)
    if response.status_code == 403:
        raise ConnectionError('API response: {}'.format(response.status_code))
    response_json = response.json()
    return response_json

@sleep_and_retry
@on_exception(expo, ConnectionError, max_tries=10)
@limits(calls=20, period=2)
def call_class_graph_api(clz_id, src):
    api_url = '{}/rxclass/classGraph.json?classId={}&source={}'.format(
        base_api_url, clz_id, src)
    response = requests.get(api_url)
    if response.status_code == 403:
        raise ConnectionError('API response: {}'.format(response.status_code))
    response_json = response.json()
    return response_json

def get_new_relationship(rxcui):
    all_clz = call_class_api(rxcui)
    if all_clz:
        rx_class_info = all_clz['rxclassDrugInfoList']['rxclassDrugInfo']

        src_clz_relationship = dict()
        for concept in rx_class_info:
            if concept['rxclassMinConceptItem']['classType'] not in src_clz_relationship:
                src_clz_relationship[concept['rxclassMinConceptItem']['classType']] = dict()
            if concept['rela'] not in src_clz_relationship[concept['rxclassMinConceptItem']['classType']]:
                src_clz_relationship[concept['rxclassMinConceptItem']['classType']][concept['rela']] = set()
            src_clz_relationship[concept['rxclassMinConceptItem']['classType']][concept['rela']].add(
                concept['rxclassMinConceptItem']['classId'])

        src_relationship = dict()
        for src, rela_ids in src_clz_relationship.items():
            if src not in src_relationship:
                src_relationship[src] = dict()
            for rela, clz_ids in rela_ids.items():
                for clz_id in clz_ids:
                    response_json = call_class_graph_api(clz_id, src)
                    if response_json['rxclassGraph']:
                        if rela not in src_relationship[src]:
                            src_relationship[src][rela] = set()
                        src_relationship[src][rela].add(
                            response_json['rxclassGraph']['rxclassMinConceptItem'][0]['className'])
                        if len(response_json['rxclassGraph']['rxclassMinConceptItem']) > 1:
                            for concept in response_json['rxclassGraph']['rxclassMinConceptItem'][1:]:
                                concept_name = concept['className']
                                src_relationship[src][rela].add(concept_name)

        src_relationship_lists = dict()
        for src, relas in src_relationship.items():
            for rela, concept in relas.items():
                if not rela:
                    comb_src = src
                else:
                    comb_src = '_'.join([src, rela])
                src_relationship_lists[comb_src] = ';'.join(concept)

        new_relationship = pd.DataFrame(src_relationship_lists, index=[0])
    else:
        new_relationship = pd.DataFrame()
    new_relationship['Best RxNorm Id'] = rxcui
    return new_relationship

In [ ]:
relationship_df = pd.DataFrame()  

In [296]:
for rxcui in set(input_df['Best RxNorm Id']):
    if rxcui:
        if rxcui not in relationship_df['Best RxNorm Id']:
            print('Working on {}'.format(rxcui))
            new_relationship = get_new_relationship(rxcui)
            relationship_df = pd.concat([relationship_df, new_relationship], ignore_index=True)

Working on 301543
Working on 2479830
Working on 91818
Working on 1348
Working on 398335
Working on 11251
Working on 203029
Working on 1234386
Working on 203345
Working on 756905
Working on 3640
Working on 10324
Working on 33738
Working on 8787
Working on 10582
Working on 11253
Working on 638596
Working on 1114038
Working on 7258
Working on 1244014
Working on 321988
Working on 209468
Working on 2598
Working on 11248
Working on 1234482
Working on 5640
Working on 1664316
Working on 153010
Working on 214559
Working on 151392
Working on 4821
Working on 1160453
Working on 6809
Working on 203302
Working on 435
Working on 1191
Working on 197784
Working on 202433
Working on 26225
Working on 495430
Working on 1895
Working on 7514
Working on 4301
Working on 1588
Working on 25480
Working on 29046
Working on 1790650
Working on 50166
Working on 1876402
Working on 28889
Working on 152061
Working on 18993
Working on 83373
Working on 5032
Working on 18631
Working on 11416
Working on 226084
Working on 2

In [297]:
relationship_df

,MESHPA,ATC1-4,DISPOS_isa_disposition,STRUCT_isa_structure,CHEM_has_ingredient,DISEASE_ci_with,DISEASE_may_treat,DISEASE_may_prevent,MOA_ci_moa,MOA_has_moa,...,TC_has_tc,PK_has_pk,THERAP_isa_therapeutic,CHEM_ci_chemclass,PK_site_of_metabolism,PE_ci_pe,DISEASE_induces,CHEM_has_chemical_structure,DISEASE_may_diagnose,SCHEDULE_has_schedule
0,Adrenergic Agonists;Glucocorticoids;Respirator...,RESPIRATORY SYSTEM;DERMATOLOGICALS;Adrenergics...,Selective beta-2 agonist-containing product;Ad...,Steroid-containing product;Phenol structure-co...,Ethanolamines;Salmeterol Xinafoate;Androstadie...,Chemically-Induced Disorders;Drug-Related Side...,"Hypersensitivity, Immediate;Lung Diseases;Nose...","Nasal Polyps;Polyps;Hypersensitivity, Immediat...",Monoamine Oxidase Inhibitors;Enzyme Interactio...,Adrenergic Agonists;Hormone Receptor Interacti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Recombinant deoxyribonucleic acid-containing p...,Viral Vaccines;Biological Products;Complex Mix...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Autonomic Agents;Peripheral Nervous System Age...,Piperazine derivatives;Anatomical Therapeutic ...,Histamine H1 receptor antagonist-containing pr...,Piperazine derivative-containing product;Piper...,"Hydrocarbons;Hydrocarbons, Cyclic;Benzene Deri...",Chemically-Induced Disorders;Drug Hypersensiti...,Motion Sickness;Labyrinth Diseases;Nervous Sys...,"Motion Sickness;Pathological Conditions, Signs...",NaN,Cholinergic Muscarinic Receptor Interactions;C...,...,Gastrointestinal Agent;Central Nervous System ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Glucocorticoids;Respiratory System Agents;Ther...,RESPIRATORY SYSTEM;DERMATOLOGICALS;Glucocortic...,NaN,Steroid-containing product;Glucocorticoid-cont...,Pregnadienetriols;Substances;Steroids;Beclomet...,Asthma;Hypersensitivity;Chemically-Induced Dis...,Hypersensitivity;Infections;Rhinitis;Otorhinol...,Asthma;Nasal Polyps;Polyps;Hypersensitivity;No...,NaN,Hormone Receptor Interactions;Corticosteroid H...,...,NaN,Renal Excretion;Fecal Excretion;Pharmacokineti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Histamine Antagonists;MeSH Pharmacologic Actio...,Piperazine derivatives;Anatomical Therapeutic ...,Histamine H1 receptor antagonist-containing pr...,NaN,Piperazines;Cetirizine;Substances;Heterocyclic...,Kidney Diseases;Urogenital Diseases;Renal Insu...,"Hypersensitivity, Immediate;Nose Diseases;Skin...",NaN,NaN,Histamine Receptor Interactions;Histamine H1 R...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,Protective Agents;MeSH Pharmacologic Actions (...,"Organic acids;Ascorbic acid (vitamin C), plain...",NaN,Carbohydrate-containing product;Medicinal prod...,"Carbohydrates;Carboxylic Acids;Acids, Acyclic;...",Chemically-Induced Disorders;Drug Hypersensiti...,Hemic and Lymphatic Diseases;Avitaminosis;Tyro...,NaN,NaN,Oxidation-Reduction Activity;Free Radical Scav...,...,NaN,NaN,NaN,NaN,NaN,NaN,Kidney Diseases;Urogenital Diseases;Female Uro...,"Carbohydrates;Carboxylic Acids;Acids, Acyclic;...",NaN,NaN
134,Protective Agents;Antiparkinson Agents;Therape...,NERVOUS SYSTEM;Dopamine agonists;DOPAMINERGIC ...,Dopamine receptor agonist-containing product;M...,NaN,"Thiazoles;Heterocyclic Compounds, 2-Ring;Azole...",Chemically-Induced Disorders;Drug Hypersensiti...,Parkinson Disease;Basal Ganglia Diseases;Prote...,NaN,NaN,Dopamine Agonists;Mechanism of Action (MoA);G-...,...,NaN,NaN,Medicinal product categorized by therapeutic r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,Vitamin B Complex;Radiation-Sensitizing Agents...,CICATRIZANTS;PREPARATIONS FOR TREATMENT OF WOU...,NaN,Nitrogen and/or nitrogen compound-containing p...,Thiamine;beta-Alanine;Nicotinic Acids;Heterocy...,Vascular Diseases;Pathologic Processes;Chemica...,Avitaminosis;Dyslipidemias;Chemically-Induced ...,Avitaminosis;Deficiency Diseases;Arterial Occl...,NaN,Enzyme Activators;Enzyme Interactions;Mechanis...,...,NaN

In [300]:
output_df = input_df.merge(relationship_df, how='outer', on='Best RxNorm Id')

In [301]:
output_df

,Unnamed: 0,ID,Event,Item,Dictionary,Verbatim Term,Preferred Term,RXCUI,Drug Name,Normalized RxNorm Id,...,TC_has_tc,PK_has_pk,THERAP_isa_therapeutic,CHEM_ci_chemclass,PK_site_of_metabolism,PE_ci_pe,DISEASE_induces,CHEM_has_chemical_structure,DISEASE_may_diagnose,SCHEDULE_has_schedule
0,0,1,Concomitant Medications,Generic Name,RxNorm,escitalopram,Escitalopram 20 MG,353383.0,Escitalopram 20 MG,321988.0,...,NaN,NaN,Antidepressant agent;Medicinal product categor...,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,Concomitant Medications,Generic Name,RxNorm,escitalopram,Escitalopram 20 MG,353383.0,Escitalopram 20 MG,321988.0,...,NaN,NaN,Antidepressant agent;Medicinal product categor...,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN,NaN,NaN,NaN,NaN
2,101,30,Concomitant Medications,Generic Name,RxNorm,escitalopram,Escitalopram,321988.0,Escitalopram,321988.0,...,NaN,NaN,Antidepressant agent;Medicinal product categor...,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN,NaN,NaN,NaN,NaN
3,101,30,Concomitant Medications,Generic Name,RxNorm,escitalopram,Escitalopram,321988.0,Escitalopram,321988.0,...,NaN,NaN,Antidepressant agent;Medicinal product categor...,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,Concomitant Medications,Generic Name,RxNorm,mirena,Mirena,152061.0,Mirena,152061.0,...,NaN,NaN,Medicinal product categorized by therapeutic r...,NaN,NaN,Organ System Specific Effects;Cardiovascular A...,"Urogenital Diseases;Life Phases, Behavior Mech...",Gonadal Hormones;Progesterone Congeners;Substa...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,147,43,Concomitant Medications,Generic Name,RxNorm,Ondansetron,Ondansetron,26225.0,Ondansetron,26225.0,...,NaN,NaN,Medicinal product categorized by therapeutic r...,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN,NaN,NaN,NaN,NaN
273,149,43,Concomitant Medications,Generic Name,RxNorm,ofloxacin otic,Ofloxacin Otic Solution,377450.0,Ofloxacin Otic Solution,1160453.0,...,Anti-infective Agent;Antimicrobial Agent;Thera...,NaN,Antiinfective agent;Fluoroquinolone antibacter...,NaN,NaN,NaN,NaN,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN
274,149,43,Concomitant Medications,Generic Name,RxNorm,ofloxacin otic,Ofloxacin Otic Solution,377450.0,Ofloxacin Otic Solution,1160453.0,...,Anti-infective Agent;Antimicrobial Agent;Thera...,NaN,Antiinfective agent;Fluoroquinolone antibacter...,NaN,NaN,NaN,NaN,"Heterocyclic Compounds, 2-Ring;Heterocyclic Co...",NaN,NaN
275,153,45,Concomitant Medications,Generic Name,RxNorm,Zinc,Zinc,11416.0,Zinc,11416.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
output_df.to_csv('../data/conmed_example_data_with_hierarchy.csv')